In [2]:
import pandas as pd
import numpy as np

# --- 1. การตั้งค่า (Configuration) ---
input_file = 'zeus_dataset_raw.csv'
output_file = 'zeus_dataset_cleaned.csv'

print(f"กำลังอ่านไฟล์ {input_file} ...")
df = pd.read_csv(input_file)
print(f"จำนวนข้อมูลเริ่มต้น: {len(df)} แถว")

# --- 2. ลบ Timezone ผิด และ Station ID ว่าง ---
if 'timezone' in df.columns:
    df = df[df['timezone'].str.strip() != 'Africa/Malabo']
if 'station_id' in df.columns:
    df = df.dropna(subset=['station_id']) # ลบแถวที่ไม่มีชื่อสถานี

# --- [ใหม่] 2.5 จัดการเวลาซ้ำ (Duplicate Time) ---
# แปลงเป็น datetime ก่อน
df['datetime'] = pd.to_datetime(df['datetime'])

# ลบเวลาที่ซ้ำกัน (เก็บแถวสุดท้ายไว้)
rows_before = len(df)
df = df.drop_duplicates(subset=['datetime'], keep='last')
print(f"ลบข้อมูลเวลาซ้ำออก: {rows_before - len(df)} แถว")

# เรียงลำดับเวลาให้ชัวร์
df = df.sort_values('datetime')

# --- 3. จัดการค่า Error Code (-999) และพิกัด 0 ---
df = df.replace(-999, np.nan)
if 'lat' in df.columns:
    df['lat'] = df['lat'].replace(0, np.nan)
    df['long'] = df['long'].replace(0, np.nan)

# แปลงตัวเลข
numeric_cols = ['temp', 'humidity', 'pressure', 'rain', 'uv', 'co2', 'pm25']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# --- 4. จัดการ Outliers (Z-score > 3) ---
outlier_check_cols = ['temp', 'humidity', 'pressure', 'co2', 'pm25', 'uv']
for col in outlier_check_cols:
    if col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        if std == 0: continue
        z_scores = np.abs((df[col] - mean) / std)
        df.loc[z_scores > 3, col] = np.nan

# --- [ใหม่] 5. Resample & Interpolate (เติมช่วงเวลาที่หายไป) ---
print("5. กำลังซ่อมแซมช่วงเวลาที่ขาดหาย (Resample 5min)...")
# ตั้งเวลาเป็น Index
df = df.set_index('datetime')

# สร้างเวลาให้ครบทุก 5 นาที (ถ้าขาดให้เติม NaN)
df = df.resample('5T').asfreq()

# เติมค่าที่หายไปด้วยการลากเส้นเชื่อม (Interpolate) เฉพาะคอลัมน์ตัวเลข
# limit=12 แปลว่าถ้าหายเกิน 1 ชั่วโมง (12*5นาที) ไม่ต้องเติม (เดี๋ยวข้อมูลมั่ว)
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].interpolate(method='time', limit=12)

# ถ้ายังมีเหลือ (เช่นหัวท้าย หรือช่วงที่หายนานมากๆ) ให้เติม Mean
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# เติมคอลัมน์ที่ไม่ใช่ตัวเลข (เช่น station_id) ด้วยค่าล่าสุด (Forward Fill)
df[['station_id', 'timezone']] = df[['station_id', 'timezone']].ffill().bfill()

# รีเซ็ต Index กลับมาเป็นคอลัมน์ datetime ปกติ
df = df.reset_index()

# --- 6. บันทึกไฟล์ใหม่ ---
df.to_csv(output_file, index=False)
print("-" * 30)
print(f"ทำความสะอาดระดับเทพเสร็จสมบูรณ์! ข้อมูลใหม่มี: {len(df)} แถว")
print(f"บันทึกไฟล์เรียบร้อยที่: {output_file}")

กำลังอ่านไฟล์ zeus_dataset_raw.csv ...
จำนวนข้อมูลเริ่มต้น: 16786 แถว
ลบข้อมูลเวลาซ้ำออก: 95 แถว
5. กำลังซ่อมแซมช่วงเวลาที่ขาดหาย (Resample 5min)...


C:\Users\Boss_Reungwit\AppData\Local\Temp\ipykernel_3480\3623971529.py:58: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('5T').asfreq()


------------------------------
ทำความสะอาดระดับเทพเสร็จสมบูรณ์! ข้อมูลใหม่มี: 41224 แถว
บันทึกไฟล์เรียบร้อยที่: zeus_dataset_cleaned.csv
